## GPT2 fine tuning

In [8]:
# Install dependencies
!pip install transformers pandas numpy torch tensorboard -qq

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    pandas from https://files.pythonhosted.org/packages/cb/69/d2a9673f2fbb96f072c28f3bf1f3a249dc1f28395fb7dce98dfd0fe25885/pandas-1.1.0-cp37-cp37m-win_amd64.whl#sha256=a15835c8409d5edc50b4af93be3377b5dd3eb53517e7f785060df1f06f6da0e2:
        Expected sha256 a15835c8409d5edc50b4af93be3377b5dd3eb53517e7f785060df1f06f6da0e2
             Got        169763fb0142fa7955329adb9617e67b39cf8f1f3e67f2d01e4f539673cca41c



In [23]:
# imports 
import random, os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TextDataset, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments)

In [45]:
MODEL_NAME = 'gpt2'

### Set up Transformer neural network

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

## Load Data

### Loading Quora Data 

In [14]:
quora_data = pd.read_csv("q_quora.csv", dtype=str)
quora_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
quora_data = quora_data.loc[train['is_duplicate']=='1']
quora_data = quora_data.drop([
    'id','qid1', 'qid2','is_duplicate','Unnamed: 6', 'Unnamed: 7', 
    'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'
], axis=1)

quora_data = quora_data.reset_index(drop=True)
quora_data.columns= ['question', 'target']

quora_data = quora_data.sample(frac=1).reset_index(drop=True)
quora_data.head()

,question,target
0,What is business course?,What is a business course?
1,What is your New Year’s Resolution(s) for 2017?,What are some of your New Year's resolutions f...
2,What is the eligibility criteria for the CAT e...,What is the eligibility for CAT exam?
3,Which is your favorite movie?,What are your favorite movies and why?
4,How can I become a journalist?,What is the best way to become a journalist?


### Loading MBTI data

In [18]:
mbti_data = pd.read_csv("mbti_1.csv")
mbti_data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [22]:
personality_types = pd.unique(mbti_data['type'])
print("All Personality types")
print("========================")
print(personality_types)

All Personality types
['INFJ' 'ENTP' 'INTP' 'INTJ' 'ENTJ' 'ENFJ' 'INFP' 'ENFP' 'ISFP' 'ISTP'
 'ISFJ' 'ISTJ' 'ESTP' 'ESFP' 'ESTJ' 'ESFJ']


In [37]:
if not os.path.exists("personality_types"):
    os.mkdir("personality_types")
    
for personality_type in personality_types:
    with open(f"personality_types/{personality_type}.txt", "wb") as personality_file:
        for post in mbti_data[mbti_data["type"] == personality_type]["posts"]:
            personality_file.write(post.encode())
            personality_file.write(b"\n")

In [38]:
personality_type = "INTJ"

epochs = 4  # limit before overfitting
block_size = tokenizer.max_len

train_dataset = TextDataset(tokenizer=tokenizer, file_path=f'personality_types/{personality_type}.txt', block_size=block_size, overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

### Set up Training Args

In [40]:
training_args = TrainingArguments(
    output_dir=f'output/{personality_type}',
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=1,
    logging_steps=5,
    save_steps=0,
    seed=random.randint(0,2**32-1))

### Set up Trainer

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    prediction_loss_only=True)

You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


### Training neural network

In [43]:
trainer.train()

### Model testing

In [44]:
# prepare input
start_with_bos = '<|endoftext|> ' + Text
stop_token = '<|endoftext|>'
encoded_prompt = tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

NameError: name 'start' is not defined

In [ ]:
# prediction
output_sequences = trainer.model.generate(
    input_ids=encoded_prompt,
    max_length=160,
    min_length=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=20
)

In [4]:




generated_sequences = []

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    limit = text.find(stop_token, 1)
    text = text[: limit if limit != -1 else None]

    # we also remove new lines which happen due to pre-trained gpt-2 and twitter scraping
    limit = text.find('\n')
    text = text[: limit if limit != -1 else None]
    generated_sequence = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
    generated_sequences.append(clean_prediction(generated_sequence))

for i, g in enumerate(generated_sequences):
    predictions.append([start, ' '.join([start, g])])

# log predictions
print({'examples': wandb.Table(data=predictions, columns=['Input', 'Prediction'])})
